In [5]:
import os 
import shutil 
from math import ceil

In [6]:
path=os.path.abspath(os.path.dirname('__file__'))

<font color=red> <font size=2>**Create a train and test folder** </font>

In [3]:
dossier=os.path.join(path,'Imgs') # folder which contain all images .
tr='{}/{}'.format(path,'train')
ts='{}/{}'.format(path,'test')
if os.path.exists(tr):
   shutil.rmtree(tr)                  
os.mkdir(tr)
if os.path.exists(ts):
   shutil.rmtree(ts)
os.mkdir(ts)
seuil=0.7
for dos in os.listdir(dossier):
    label=os.path.join(dossier,dos)
    l=len(os.listdir(label))
    r=ceil(seuil*l)
    dos_tr='{}/{}'.format(tr,dos)
    dos_ts='{}/{}'.format(ts,dos)
    if os.path.exists(dos_tr):
       shutil.rmtree(dos_tr)
    if os.path.exists(dos_ts):
       shutil.rmtree(dos_ts)
    os.mkdir(dos_tr)
    os.mkdir(dos_ts) 
    for file in os.listdir(label)[:r]:
        image=os.path.join(label,file)
        shutil.copy(image,dos_tr)
    for file in os.listdir(label)[r:]:
        image=os.path.join(label,file)
        shutil.copy(image,dos_ts)

# Datas Augmentation

In [7]:
from keras.preprocessing.image import ImageDataGenerator
batch_size=16
#prepare data augmentation generation
tr='{}/{}'.format(path,'train')
ts='{}/{}'.format(path,'test')
train_datagen=ImageDataGenerator(rescale=1/255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1/255)

In [8]:
#training data aufmentation 
train_generator=train_datagen.flow_from_directory(tr,target_size=(224,224),batch_size=batch_size,\
                                                 class_mode='categorical')
#test data augmentation
test_generator=test_datagen.flow_from_directory(ts,target_size=(224,224),batch_size=batch_size,\
                                                 class_mode='categorical')

Found 14458 images belonging to 120 classes.
Found 6122 images belonging to 120 classes.


# CNN Aproach

<font color=red><font size=3> **Model implementation ** </font>

In [9]:
from keras.applications import VGG16
from keras.layers import Dense,Flatten,GlobalAveragePooling2D,Dropout,Conv2D,MaxPool2D
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import load_img,img_to_array

In [10]:
# import VGG16 architecture without fully connected layers.
model=VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))
x1=model.output    # recuperate the output of our model.
#Add new fully connected layers to determinate dog's breed.
x1=Flatten()(x1)
y1=Dense(210,activation='relu')(x1)
y1=Dense(200,activation='relu')(y1)
prediction=Dense(120,activation='softmax')(y1)
#Define the new model
new_model=Model(inputs=model.input,outputs=prediction)

In [11]:
#save the weight of the fifth first layers and retrain the rest of the other layers.
for layer in model.layers[:5]:
    layer.trainable=False

In [13]:
from keras import optimizers
new_model.compile(loss='categorical_crossentropy',optimizer=optimizers.SGD(lr=0.001,momentum=0.9),metrics=['accuracy'])

In [14]:
new_model.fit_generator(train_generator,steps_per_epoch=batch_size,epochs=600,validation_data=test_generator\
                       ,validation_steps=batch_size,verbose=2)


Epoch 1/600
 - 226s - loss: 4.8983 - accuracy: 0.0117 - val_loss: 4.8103 - val_accuracy: 0.0117
Epoch 2/600
 - 289s - loss: 4.8168 - accuracy: 0.0039 - val_loss: 4.7647 - val_accuracy: 0.0156
Epoch 3/600
 - 479s - loss: 4.7898 - accuracy: 0.0117 - val_loss: 4.7345 - val_accuracy: 0.0117
Epoch 4/600
 - 477s - loss: 4.7844 - accuracy: 0.0156 - val_loss: 4.7909 - val_accuracy: 0.0117
Epoch 5/600
 - 479s - loss: 4.7939 - accuracy: 0.0195 - val_loss: 4.7958 - val_accuracy: 0.0078
Epoch 6/600
 - 478s - loss: 4.7831 - accuracy: 0.0156 - val_loss: 4.7827 - val_accuracy: 0.0156
Epoch 7/600
 - 478s - loss: 4.7863 - accuracy: 0.0195 - val_loss: 4.7871 - val_accuracy: 0.0000e+00
Epoch 8/600
 - 476s - loss: 4.7853 - accuracy: 0.0195 - val_loss: 4.7625 - val_accuracy: 0.0039
Epoch 9/600
 - 476s - loss: 4.7862 - accuracy: 0.0078 - val_loss: 4.8067 - val_accuracy: 0.0078
Epoch 10/600
 - 473s - loss: 4.7739 - accuracy: 0.0234 - val_loss: 4.8080 - val_accuracy: 0.0117
Epoch 11/600
 - 472s - loss: 4.790

 - 440s - loss: 4.7856 - accuracy: 0.0000e+00 - val_loss: 4.7326 - val_accuracy: 0.0039
Epoch 84/600
 - 439s - loss: 4.7877 - accuracy: 0.0195 - val_loss: 4.7881 - val_accuracy: 0.0195
Epoch 85/600
 - 439s - loss: 4.7842 - accuracy: 0.0117 - val_loss: 4.8132 - val_accuracy: 0.0078
Epoch 86/600
 - 440s - loss: 4.7821 - accuracy: 0.0117 - val_loss: 4.7430 - val_accuracy: 0.0117
Epoch 87/600
 - 439s - loss: 4.7730 - accuracy: 0.0273 - val_loss: 4.7613 - val_accuracy: 0.0312
Epoch 88/600
 - 438s - loss: 4.7876 - accuracy: 0.0078 - val_loss: 4.8133 - val_accuracy: 0.0195
Epoch 89/600
 - 431s - loss: 4.7768 - accuracy: 0.0040 - val_loss: 4.7838 - val_accuracy: 0.0078
Epoch 90/600
 - 440s - loss: 4.7819 - accuracy: 0.0039 - val_loss: 4.7548 - val_accuracy: 0.0000e+00
Epoch 91/600
 - 440s - loss: 4.7853 - accuracy: 0.0039 - val_loss: 4.7704 - val_accuracy: 0.0039
Epoch 92/600
 - 441s - loss: 4.7879 - accuracy: 0.0039 - val_loss: 4.7672 - val_accuracy: 0.0078
Epoch 93/600
 - 442s - loss: 4.7827

Epoch 167/600
 - 440s - loss: 4.7803 - accuracy: 0.0117 - val_loss: 4.8087 - val_accuracy: 0.0234
Epoch 168/600
 - 438s - loss: 4.7882 - accuracy: 0.0078 - val_loss: 4.7982 - val_accuracy: 0.0160
Epoch 169/600
 - 438s - loss: 4.7613 - accuracy: 0.0078 - val_loss: 4.7128 - val_accuracy: 0.0156
Epoch 170/600
 - 438s - loss: 4.7906 - accuracy: 0.0234 - val_loss: 4.8058 - val_accuracy: 0.0352
Epoch 171/600
 - 437s - loss: 4.7474 - accuracy: 0.0078 - val_loss: 4.8422 - val_accuracy: 0.0273
Epoch 172/600
 - 441s - loss: 4.7404 - accuracy: 0.0156 - val_loss: 4.7416 - val_accuracy: 0.0078
Epoch 173/600
 - 440s - loss: 4.7674 - accuracy: 0.0156 - val_loss: 4.7088 - val_accuracy: 0.0117
Epoch 174/600
 - 441s - loss: 4.7437 - accuracy: 0.0352 - val_loss: 4.7806 - val_accuracy: 0.0234
Epoch 175/600
 - 441s - loss: 4.7352 - accuracy: 0.0195 - val_loss: 4.8056 - val_accuracy: 0.0117
Epoch 176/600
 - 439s - loss: 4.7610 - accuracy: 0.0078 - val_loss: 4.8077 - val_accuracy: 0.0508
Epoch 177/600
 - 440

Epoch 251/600
 - 442s - loss: 3.7425 - accuracy: 0.1094 - val_loss: 4.4803 - val_accuracy: 0.1406
Epoch 252/600
 - 441s - loss: 3.7590 - accuracy: 0.1016 - val_loss: 3.8911 - val_accuracy: 0.1289
Epoch 253/600
 - 439s - loss: 3.6942 - accuracy: 0.1445 - val_loss: 3.4003 - val_accuracy: 0.0977
Epoch 254/600
 - 440s - loss: 3.6220 - accuracy: 0.1523 - val_loss: 4.1868 - val_accuracy: 0.1172
Epoch 255/600
 - 440s - loss: 3.6652 - accuracy: 0.1289 - val_loss: 3.4371 - val_accuracy: 0.1484
Epoch 256/600
 - 441s - loss: 3.5169 - accuracy: 0.1680 - val_loss: 3.2882 - val_accuracy: 0.1680
Epoch 257/600
 - 440s - loss: 3.7203 - accuracy: 0.1211 - val_loss: 3.8999 - val_accuracy: 0.1914
Epoch 258/600
 - 440s - loss: 3.6591 - accuracy: 0.1367 - val_loss: 3.2820 - val_accuracy: 0.1602
Epoch 259/600
 - 440s - loss: 3.4774 - accuracy: 0.1406 - val_loss: 3.4519 - val_accuracy: 0.1758
Epoch 260/600
 - 433s - loss: 3.4269 - accuracy: 0.1600 - val_loss: 3.1113 - val_accuracy: 0.1719
Epoch 261/600
 - 440

Epoch 335/600
 - 436s - loss: 2.1266 - accuracy: 0.3984 - val_loss: 1.9909 - val_accuracy: 0.3867
Epoch 336/600
 - 438s - loss: 2.2223 - accuracy: 0.4375 - val_loss: 2.0640 - val_accuracy: 0.4720
Epoch 337/600
 - 441s - loss: 2.1881 - accuracy: 0.3906 - val_loss: 1.7055 - val_accuracy: 0.4141
Epoch 338/600
 - 440s - loss: 1.9124 - accuracy: 0.4609 - val_loss: 1.6412 - val_accuracy: 0.4297
Epoch 339/600
 - 440s - loss: 1.9354 - accuracy: 0.4570 - val_loss: 1.8704 - val_accuracy: 0.4062
Epoch 340/600
 - 439s - loss: 1.9853 - accuracy: 0.4219 - val_loss: 3.7323 - val_accuracy: 0.4141
Epoch 341/600
 - 440s - loss: 2.0389 - accuracy: 0.4297 - val_loss: 2.2367 - val_accuracy: 0.3750
Epoch 342/600
 - 440s - loss: 1.7939 - accuracy: 0.4961 - val_loss: 2.0255 - val_accuracy: 0.2891
Epoch 343/600
 - 440s - loss: 1.9395 - accuracy: 0.4297 - val_loss: 1.5375 - val_accuracy: 0.3984
Epoch 344/600
 - 441s - loss: 2.0360 - accuracy: 0.3828 - val_loss: 2.7646 - val_accuracy: 0.3594
Epoch 345/600
 - 442

Epoch 419/600
 - 440s - loss: 1.5664 - accuracy: 0.5117 - val_loss: 1.8630 - val_accuracy: 0.4648
Epoch 420/600
 - 440s - loss: 1.5260 - accuracy: 0.5391 - val_loss: 2.4448 - val_accuracy: 0.4961
Epoch 421/600
 - 441s - loss: 1.6534 - accuracy: 0.4766 - val_loss: 2.1318 - val_accuracy: 0.5078
Epoch 422/600
 - 442s - loss: 1.5227 - accuracy: 0.5742 - val_loss: 2.0719 - val_accuracy: 0.4453
Epoch 423/600
 - 441s - loss: 1.6831 - accuracy: 0.5312 - val_loss: 2.1177 - val_accuracy: 0.4922
Epoch 424/600
 - 441s - loss: 1.4305 - accuracy: 0.5508 - val_loss: 1.3694 - val_accuracy: 0.5078
Epoch 425/600
 - 440s - loss: 1.3264 - accuracy: 0.6172 - val_loss: 1.3807 - val_accuracy: 0.5039
Epoch 426/600
 - 444s - loss: 1.4031 - accuracy: 0.5938 - val_loss: 1.8631 - val_accuracy: 0.4688
Epoch 427/600
 - 441s - loss: 1.4787 - accuracy: 0.5742 - val_loss: 1.9419 - val_accuracy: 0.4258
Epoch 428/600
 - 442s - loss: 1.7068 - accuracy: 0.5078 - val_loss: 1.8726 - val_accuracy: 0.5391
Epoch 429/600
 - 439

Epoch 503/600
 - 438s - loss: 1.4008 - accuracy: 0.5898 - val_loss: 1.6186 - val_accuracy: 0.4880
Epoch 504/600
 - 440s - loss: 1.2095 - accuracy: 0.6289 - val_loss: 2.4326 - val_accuracy: 0.5273
Epoch 505/600
 - 440s - loss: 1.2798 - accuracy: 0.5977 - val_loss: 1.7035 - val_accuracy: 0.5117
Epoch 506/600
 - 440s - loss: 1.1951 - accuracy: 0.6406 - val_loss: 1.8364 - val_accuracy: 0.5156
Epoch 507/600
 - 442s - loss: 1.3627 - accuracy: 0.5586 - val_loss: 1.5450 - val_accuracy: 0.5039
Epoch 508/600
 - 441s - loss: 1.3569 - accuracy: 0.5664 - val_loss: 1.9261 - val_accuracy: 0.4688
Epoch 509/600
 - 441s - loss: 1.2889 - accuracy: 0.6172 - val_loss: 1.7734 - val_accuracy: 0.5273
Epoch 510/600
 - 440s - loss: 1.1500 - accuracy: 0.6680 - val_loss: 1.1540 - val_accuracy: 0.5469
Epoch 511/600
 - 440s - loss: 0.9685 - accuracy: 0.6914 - val_loss: 2.2141 - val_accuracy: 0.5469
Epoch 512/600
 - 440s - loss: 1.1387 - accuracy: 0.6367 - val_loss: 1.6184 - val_accuracy: 0.5273
Epoch 513/600
 - 443

Epoch 587/600
 - 440s - loss: 0.8558 - accuracy: 0.7305 - val_loss: 2.0091 - val_accuracy: 0.5391
Epoch 588/600
 - 440s - loss: 0.9115 - accuracy: 0.7227 - val_loss: 1.6466 - val_accuracy: 0.5312
Epoch 589/600
 - 441s - loss: 1.0404 - accuracy: 0.7148 - val_loss: 1.6526 - val_accuracy: 0.5273
Epoch 590/600
 - 439s - loss: 1.0585 - accuracy: 0.6602 - val_loss: 1.1338 - val_accuracy: 0.5703
Epoch 591/600
 - 441s - loss: 0.9966 - accuracy: 0.7070 - val_loss: 1.7101 - val_accuracy: 0.5117
Epoch 592/600
 - 441s - loss: 0.8213 - accuracy: 0.7422 - val_loss: 1.0647 - val_accuracy: 0.5742
Epoch 593/600
 - 440s - loss: 0.9973 - accuracy: 0.7266 - val_loss: 1.1267 - val_accuracy: 0.6016
Epoch 594/600
 - 440s - loss: 0.9733 - accuracy: 0.6836 - val_loss: 1.9060 - val_accuracy: 0.5273
Epoch 595/600
 - 441s - loss: 1.0445 - accuracy: 0.7070 - val_loss: 1.4681 - val_accuracy: 0.5586
Epoch 596/600
 - 442s - loss: 1.0622 - accuracy: 0.7109 - val_loss: 1.3571 - val_accuracy: 0.5508
Epoch 597/600
 - 440

In [15]:
new_model.evaluate_generator(test_generator)

[1.6316931247711182, 0.5772623419761658]